In [1]:
pip install librosa numpy soundfile

In [5]:
import numpy as np
import librosa
import soundfile as sf

def spectrogram_to_audio(spectrogram, sr=22050, n_iter=32, hop_length=512, win_length=None):
    """
    Convert a magnitude spectrogram back to an audio signal using the Griffin-Lim algorithm.

    Parameters:
    - spectrogram: np.ndarray, the input magnitude spectrogram.
    - sr: int, the sample rate of the original audio signal.
    - n_iter: int, the number of iterations for phase reconstruction using Griffin-Lim.
    - hop_length: int, the hop length used for the STFT.
    - win_length: int or None, the window length used for the STFT. If None, it defaults to `n_fft`.

    Returns:
    - audio_signal: np.ndarray, the reconstructed audio signal.
    """
    # Griffin-Lim algorithm to estimate phase given only the magnitude spectrogram
    audio_signal = librosa.griffinlim(spectrogram, n_iter=n_iter, hop_length=hop_length, win_length=win_length)
    return audio_signal

def save_audio_to_wav(audio_signal, output_file_path, sr=22050):
    """
    Save an audio signal to a WAV file.

    Parameters:
    - audio_signal: np.ndarray, the audio signal to save.
    - output_file_path: str, the path to the output WAV file.
    - sr: int, the sample rate of the audio signal.
    """
    sf.write(output_file_path, audio_signal, sr)

# Example usage:
if __name__ == "__main__":
    # Load an example spectrogram (replace this with your spectrogram)
    y, sr = librosa.load(librosa.example('trumpet'), sr=None)
    S = np.abs(librosa.stft(y))

    # Convert the spectrogram to audio
    audio_signal = spectrogram_to_audio(S, sr=sr)

    # Save the reconstructed audio to a WAV file
    save_audio_to_wav(audio_signal, 'reconstructed_audio.wav', sr=sr)


In [7]:
import numpy as np
import librosa
import soundfile as sf
import matplotlib.pyplot as plt

def image_to_spectrogram(image_path):
    """
    Load an image and convert it into a spectrogram.

    Parameters:
    - image_path: str, the path to the image file.

    Returns:
    - spectrogram: np.ndarray, the converted spectrogram.
    """
    image = plt.imread(image_path)
    # Assuming the image is in grayscale, normalize and convert to spectrogram scale
    normalized_image = (image - image.min()) / (image.max() - image.min())
    # Convert to amplitude spectrogram (assuming image represents dB scale)
    spectrogram = librosa.db_to_amplitude(normalized_image * 80)
    return spectrogram

def spectrogram_to_audio(spectrogram, sr=22050, n_iter=32, hop_length=512, win_length=None):
    """
    Convert a spectrogram back to an audio signal using the Griffin-Lim algorithm.
    """
    # Determine n_fft as twice the spectrogram rows minus 2
    n_fft = (spectrogram.shape[0] - 1) * 2

    # Adjust hop_length if not specified
    if hop_length is None:
        hop_length = n_fft // 4

    audio_signal = librosa.griffinlim(spectrogram, n_iter=n_iter, hop_length=hop_length, win_length=win_length)
    return audio_signal

def save_audio_to_wav(audio_signal, output_file_path, sr=22050):
    """
    Save an audio signal to a WAV file.
    """

    sf.write(output_file_path, audio_signal, sr)

# Example usage:
if __name__ == "__main__":
    image_path = '/content/test_gray.jpeg'  # Path to your spectrogram image
    output_path = '/content/reconstructed_audio.wav'

    # Convert the image to a spectrogram
    spectrogram = image_to_spectrogram(image_path)

    # Convert the spectrogram to audio
    audio_signal = spectrogram_to_audio(spectrogram)

    # Save the reconstructed audio to a WAV file
    save_audio_to_wav(audio_signal, output_path)


In [9]:
import numpy as np
import librosa
import soundfile as sf
import matplotlib.pyplot as plt

def image_to_spectrogram(image_path):
    # Load the image
    image = plt.imread(image_path)
    if len(image.shape) == 3:  # If the image is RGB
        # Convert to grayscale
        image = np.mean(image[..., :3], axis=2)
    # Normalize the grayscale image to convert pixel values to spectrogram magnitudes
    min_level_db = -100
    ref_level_db = 20
    normalized_image = (image - np.min(image)) / (np.max(image) - np.min(image))
    # Convert to amplitude spectrogram
    spectrogram = librosa.db_to_amplitude(normalized_image * (ref_level_db - min_level_db) + min_level_db)
    return spectrogram


def griffin_lim(spectrogram, sr=22050, n_iter=100, hop_length=512, win_length=1024):
    # Perform the Griffin-Lim algorithm to reconstruct the audio signal from the amplitude spectrogram
    n_fft = (spectrogram.shape[0] - 1) * 2
    audio = librosa.griffinlim(spectrogram, n_iter=n_iter, hop_length=hop_length, win_length=win_length, n_fft=n_fft)
    return audio

def save_audio(audio, sr, file_path):
    # Save the audio signal to a file
    sf.write(file_path, audio, sr)

# File paths
image_path = '/content/test_gray.jpeg'
output_wav_path = '/content/output_audio.wav'

# Convert the image to a spectrogram
spectrogram = image_to_spectrogram(image_path)

# Perform phase reconstruction using the Griffin-Lim algorithm
audio_signal = griffin_lim(spectrogram)

# Save the audio signal to a .wav file
save_audio(audio_signal, sr=22050, file_path=output_wav_path)


In [13]:
import numpy as np
import librosa
import soundfile as sf
import matplotlib.pyplot as plt

def image_to_spectrogram(image_path, n_fft, hop_length):
    # Load the image (possibly in color)
    image = plt.imread(image_path)
    if len(image.shape) == 3 and image.shape[2] == 3:  # Check if the image is in color
        # Convert to grayscale by averaging the RGB channels
        image = np.mean(image, axis=-1)
    # Assuming the grayscale image represents a linear magnitude spectrogram in dB scale
    image = np.clip(image, 0, 1)  # Ensure the values are within [0, 1] range
    spectrogram = librosa.db_to_amplitude(image * 80)  # Assuming the image represents dB values
    return spectrogram

def griffin_lim(spectrogram, sr, hop_length, win_length, n_iter=100):
    # Reconstruct the audio signal from the amplitude spectrogram
    n_fft = (spectrogram.shape[0] - 1) * 2
    audio_reconstructed = librosa.griffinlim(spectrogram, n_iter=n_iter, hop_length=hop_length, win_length=win_length, n_fft=n_fft)
    return audio_reconstructed

def save_audio_to_wav(audio, sr, file_path):
    # Save the audio signal to a WAV file
    sf.write(file_path, audio, sr)

# Adjust these parameters as necessary
sr = 44100  # Replace with the actual sample rate if known
n_fft = 512
hop_length = 64  # 25% of the window size
win_length = 256

# Load the spectrogram image
image_path = '/content/test_gray.jpeg'  # Update with the actual path to your spectrogram image
spectrogram = image_to_spectrogram(image_path, n_fft, hop_length)

# Perform phase reconstruction using the Griffin-Lim algorithm
audio_signal = griffin_lim(spectrogram, sr=sr, hop_length=hop_length, win_length=win_length)

# Save the audio signal to a WAV file
output_wav_path = '/content/output_audio.wav'  # Update with the desired output path
save_audio_to_wav(audio_signal, sr=sr, file_path=output_wav_path)
